# Constrained planning ⛓️

This notebook illustrates how to use constraints with airo-planner. The API does not currently support constraints, but since all underlying OMPL objects are accessible, we can seamlessly use [OMPL's constrained planning API](https://ompl.kavrakilab.org/constrainedPlanning.html).
Constrained planning is useful when, e.g., we want to keep a cup level, or keep the end-effector in a plane, etc.

## Theory and terminology

- We call the robot configuration space (also called the ambient space) $Q$
- The space $Q_{free} \subset Q$ is free of collisions
- The constraint function $F(q)$ maps the configuration space to a real-valued vector $x \in \mathrm{R}^n$, $F:Q\rightarrow\mathrm{R}^n$
- We consider the constraint satisfied when $F(q) = 0$
- The space $X \subset Q$ consists of the configurations that satisfy the constraint: $X = \{ q \in \mathcal{Q} \mid F(q) = 0 \}$

## Important remark

Currently, OMPL's pre-built wheels break constrained planning: https://github.com/ompl/ompl/issues/1228

## 1. Setup 🏗️️

### 1.1 Building our scene 🏠

We will create a scene with a UR3e arm and an obstacle that the arm needs to pass.
The constraint will be that the TCP must remain in a top down orientation at all times.

In [ ]:
import numpy as np
from airo_drake import SingleArmScene, add_floor, add_manipulator, add_meshcat, finish_build, animate_joint_trajectory, \
    time_parametrize_toppra, visualize_frame, add_wall
from airo_planner import SingleArmOmplPlanner, function_to_ompl, uniform_symmetric_joint_bounds
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from ompl import base as ob
from ompl import geometric as og
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from ur_analytic_ik_ext import ur3e

In [ ]:
DOF = 6
START_JOINTS = np.deg2rad([0, -90, 90, -90, -90, 0])
GOAL_JOINTS = np.array([2.32027433, -1.56661515, 1.58208421, -1.58626538, -1.57079633, -0.82131832])

In [ ]:
def build_scene() -> SingleArmScene:
    robot_diagram_builder = RobotDiagramBuilder()

    meshcat = add_meshcat(robot_diagram_builder)
    arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur3e", "robotiq_2f_85", static_gripper=True)
    add_floor(robot_diagram_builder)
    add_wall(robot_diagram_builder, "YZ", 0.2, 0.1, 0.05, np.array([0.0, -0.3, 0.1]))
    robot_diagram, context = finish_build(robot_diagram_builder)

    scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)

    visualize_frame(scene.meshcat, "goal_pose", ur3e.forward_kinematics(*GOAL_JOINTS.tolist()))

    collision_checker = SceneGraphCollisionChecker(
        model=scene.robot_diagram,
        robot_model_instances=[scene.arm_index, scene.gripper_index],
        edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
        env_collision_padding=0.005,
        self_collision_padding=0.005,
    )

    plant = scene.robot_diagram.plant()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositions(plant_context, scene.arm_index, START_JOINTS)
    scene.robot_diagram.ForcedPublish(context)

    return scene, collision_checker

In [ ]:
def inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> list[JointConfigurationType]:
    solutions = ur3e.inverse_kinematics(tcp_pose)
    solutions = [solution.squeeze() for solution in solutions]
    return solutions

## 1.2 Unconstrained planning 🐋

Let's see how the robot arm moves around the obstacle when the end-effector orientation is not constrained.

In [ ]:
scene, collision_checker = build_scene()

planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree, inverse_kinematics_fn=inverse_kinematics_fn)

path = planner.plan_to_joint_configuration(START_JOINTS, GOAL_JOINTS)

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)

animate_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_index, trajectory)

## 1.3 Constrained planning 🔗

### 1.3.1 Defining the constraint 🔒

As mentioned earlier, the constraint should be that the end-effector points downwards throughout the entire path.
We defined a constraint to be a function $F:Q\rightarrow\mathrm{R}^n$, and $F(q)=0$ when the constraint is satisfied.
$F$ should in general be continuous and differentiable.

In [ ]:
class TCPConstraint(ob.Constraint):
    def __init__(self):
        super(TCPConstraint, self).__init__(DOF, 1)

    def function(self, x, out):
        eef_pose = ur3e.forward_kinematics(*x.tolist())
        dotprod = eef_pose[:3, 2].dot(np.array([0., 0., -1.]))
        out[0] = 1.0 - dotprod

    # You can optionally also implement the jacobian as an optimization.

In [ ]:
def bounds_to_ompl(joint_bounds) -> ob.RealVectorBounds:
    degrees_of_freedom = len(joint_bounds[0])
    bounds = ob.RealVectorBounds(degrees_of_freedom)
    joint_bounds_lower = joint_bounds[0]
    joint_bounds_upper = joint_bounds[1]
    for i in range(degrees_of_freedom):
        bounds.setLow(i, joint_bounds_lower[i])
        bounds.setHigh(i, joint_bounds_upper[i])
    return bounds

def _state_to_ompl(state_numpy: np.ndarray, space: ob.StateSpace) -> ob.State:
    state = ob.State(space)
    for i in range(len(state_numpy)):
        state()[i] = state_numpy[i]
    return state


def _state_from_ompl(state: ob.State, n: int) -> np.ndarray:
    return np.array([state[i] for i in range(n)])

def _get_space():
    space = ob.RealVectorStateSpace(DOF)
    space.setBounds(bounds_to_ompl(uniform_symmetric_joint_bounds(6)))
    return space

space = _get_space()
constraint = TCPConstraint()
constraint_tolerance = 0.05
constraint.setTolerance(constraint_tolerance)

### 1.3.2 Constrained planning and sampling-based motion planning algorithms 🏃

In this example, we will use a projection operator to project sampled configurations onto the manifold where the constraint is satisfied.

In [ ]:
css = ob.ProjectedStateSpace(space, constraint)
csi = ob.ConstrainedSpaceInformation(css)

delta = ob.CONSTRAINED_STATE_SPACE_DELTA
lambda_ = ob.CONSTRAINED_STATE_SPACE_LAMBDA
css.setup()
css.setDelta(delta)
css.setLambda(lambda_)
simple_setup = og.SimpleSetup(csi)

planner = og.RRTConnect(simple_setup.getSpaceInformation())
simple_setup.setPlanner(planner)

is_state_valid_fn = function_to_ompl(collision_checker.CheckConfigCollisionFree, DOF)
simple_setup.setStateValidityChecker(ob.StateValidityCheckerFn(is_state_valid_fn))
simple_setup.setOptimizationObjective(ob.PathLengthOptimizationObjective(simple_setup.getSpaceInformation()))
step = float(np.deg2rad(5))
resolution = step / space.getMaximumExtent()
simple_setup.getSpaceInformation().setStateValidityCheckingResolution(resolution)

### 1.3.2 Implementing constrained planning with OMPL's Python API ⚙️

In [ ]:
scene, collision_checker = build_scene()

goal_pose = np.array([[0.0, -1.0, 0.0, 0.30],
                          [-1.0, 0.0, 0.0, -0.13],
                          [0.0, 0.0, -1.0, 0.30],
                          [0.0, 0.0, 0.0, 1.0]])
visualize_frame(scene.meshcat, "goal_pose", goal_pose)
# input("Goal pose frame is now shown.")
goal_joints_solutions = inverse_kinematics_fn(goal_pose)
print("Found", len(goal_joints_solutions), "solutions.")

goal_joints_solutions = [solution for solution in goal_joints_solutions if
                         collision_checker.CheckConfigCollisionFree(solution)]
print("Found", len(goal_joints_solutions), "solutions that are not in collision.")

print("Checking constraint function values for goal poses (0 = satisfies constraints).")
for goal_joints in goal_joints_solutions:
    out = np.empty(1)
    constraint.function(goal_joints, out)
    print(goal_joints, out, "✅" if out[0] < constraint_tolerance else "💥")

for goal_joints in goal_joints_solutions:
    simple_setup.clear()
    space = simple_setup.getStateSpace()
    start_state = _state_to_ompl(START_JOINTS, space)
    goal_state = _state_to_ompl(goal_joints, space)
    simple_setup.setStartAndGoalStates(start_state, goal_state)

    simple_setup.solve(5.0)

    if not simple_setup.haveExactSolutionPath():
        print("No exact solution found.")
        continue

    # Simplify and smooth the path, note that this conserves path validity
    simple_setup.simplifySolution()
    path_simplifier = og.PathSimplifier(simple_setup.getSpaceInformation())
    path = simple_setup.getSolutionPath()
    path_simplifier.smoothBSpline(path)

    # Extract path
    path = np.array([_state_from_ompl(path.getState(i), DOF) for i in range(path.getStateCount())])

    trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)

    animate_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_index, trajectory)